In [ ]:
import requests
import tkinter as tk
from tkinter import Toplevel, Label, Button
import webbrowser
import pyttsx3
import time
import threading
from geopy.distance import geodesic
import folium
from datetime import datetime
 
# Initialize text-to-speech engine
engine = pyttsx3.init()
engine.setProperty("rate", 150)
 
# Meal Break Timings
MEAL_TIMES = {"Breakfast": "08:00", "Lunch": "13:00", "Tea Break": "17:00", "Dinner": "20:00"}
 
# Speak message function
def speak_message(message):
    engine.say(message)
    engine.runAndWait()
 
# Open Google Maps with voice alert
def navigate_to_station(name, distance, start_lat, start_lon, dest_lat, dest_lon):
    message = f"Navigating to {name}. It is {distance} kilometers away."
    speak_message(message)
    url = f"https://www.google.com/maps/dir/{start_lat},{start_lon}/{dest_lat},{dest_lon}"
    webbrowser.open(url)
 
# Get live location (simulation)
def get_live_location():
    return {"latitude": 21.1458 + (time.time() % 0.01), "longitude": 79.0882 + (time.time() % 0.01)}
 
# Fetch fuel stations
def fetch_fuel_stations(lat, lon, radius_km=10, limit=3):
    overpass_url = "https://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    node["amenity"="fuel"](around:{radius_km * 1000},{lat},{lon});
    out body;
    """
    response = requests.get(overpass_url, params={"data": query})
    if response.status_code == 200:
        stations = [{
            "name": s.get("tags", {}).get("name", "Unknown Petrol Pump"),
            "latitude": s["lat"],
            "longitude": s["lon"],
            "brand": s.get("tags", {}).get("brand", "Unknown"),
            "distance": round(geodesic((lat, lon), (s["lat"], s["lon"])).km, 2)
        } for s in response.json().get("elements", [])]
        return sorted(stations, key=lambda x: x["distance"])[:limit]
    return []
 
# Show pop-up message
def show_popup(title, message, details, locations=None):
    popup = Toplevel(root)
    popup.title(title)
    popup.geometry("450x350")
    popup.configure(bg="#f0f0f0")
    Label(popup, text=f"🚛 {message}", font=("Arial", 14, "bold"), bg="#f0f0f0", wraplength=400).pack(pady=10)
    Label(popup, text=details, font=("Arial", 12), fg="blue", bg="#f0f0f0").pack()
    if locations:
        for location in locations:
            Button(popup, text=f"Navigate to {location['name']}",
                   command=lambda l=location: navigate_to_station(
                       l['name'], l['distance'], truck_lat, truck_lon, l['latitude'], l['longitude']
                   ),
                   bg="green", fg="white", font=("Arial", 10, "bold")).pack(pady=2)
    Button(popup, text="Close", command=popup.destroy, bg="red", fg="white", font=("Arial", 10, "bold")).pack(pady=5)
    threading.Thread(target=speak_message, args=(message,)).start()
 
# Check fuel stations
def check_fuel_stations():
    global truck_lat, truck_lon
    fuel_stations = fetch_fuel_stations(truck_lat, truck_lon, 10, 3)
    if fuel_stations:
        message = "Upcoming fuel stations nearby:"
        details = "\n".join([f"{s['name']} ({s['brand']}) - {s['distance']} km" for s in fuel_stations])
        show_popup("Fuel Alert", message, details, fuel_stations)
    root.after(1800000, check_fuel_stations)
 
# Update map
def update_map():
    global truck_lat, truck_lon
    live_location = get_live_location()
    truck_lat, truck_lon = live_location["latitude"], live_location["longitude"]
    truck_map = folium.Map(location=[truck_lat, truck_lon], zoom_start=15)
    folium.Marker([truck_lat, truck_lon], tooltip="🚛 Truck Location", icon=folium.Icon(color="blue")).add_to(truck_map)
    fuel_stations = fetch_fuel_stations(truck_lat, truck_lon, 10, 3)
    for station in fuel_stations:
        folium.Marker(
            [station['latitude'], station['longitude']],
            tooltip=station['name'],
            icon=folium.Icon(color="red")
        ).add_to(truck_map)
    truck_map.save("truck_map.html")
    if not hasattr(update_map, "map_opened"):
        webbrowser.open("truck_map.html")
        update_map.map_opened = True
    root.after(10000, update_map)
 
root = tk.Tk()
root.title("Live Truck Tracking System")
root.geometry("400x300")
 
update_map()
check_fuel_stations()
root.mainloop()